In [ ]:
# python3 -m venv env_LangChain
# env_LangChain\Scripts\activate.bat

In [42]:
from openai import OpenAI
import datetime
from config import OPENAI_API_KEY
import asyncio
import pickle

In [18]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [19]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [21]:
message = "Which is the latest free version of ChatGPT which can be used in APIs?"
response = client.chat.completions.create(
    messages=[
        {
        "role":"user",
        "content":message
        }
        ],
    model=llm_model
)
response


ChatCompletion(id='chatcmpl-8RxPm8ajR2FaGI8pkygNhaHcYbZbm', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="As an AI language model, I do not have up-to-date information on ChatGPT's latest free version available for use in APIs. It is recommended to check the ChatGPT website or documentation for the latest information on their available versions and API usage.", role='assistant', function_call=None, tool_calls=None))], created=1701673938, model='gpt-3.5-turbo-0301', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=52, prompt_tokens=25, total_tokens=77))

In [31]:
response.choices[0].message.model_dump().get("content")

"As an AI language model, I do not have up-to-date information on ChatGPT's latest free version available for use in APIs. It is recommended to check the ChatGPT website or documentation for the latest information on their available versions and API usage."

In [32]:
def get_completion(prompt):
    messages = [{"role":"user", "message": prompt}]
    response = client.chat.completions.create(
        messages=messages,
        model = llm_model
    )
    return response, response.choices[0].message.model_dump().get("content")
    

In [36]:
# import pickle
# with open("chat_response_object.pkl",mode='wb') as file:
#     pickle.dump(file=file, obj=response)

In [38]:
with open("./chat_response_object.pkl", mode="rb") as file: 
    response_read = pickle.load(file)
response_read.choices[0].message.model_dump().get("content")

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [47]:
case_text = """

                                                         
                   WP(CRL)-2013-132-J       

Case No           132 
Year              2013 
Date of Decision  07/05/2013 
Case Type :       WRIT PETITION CRIMINAL 
Decided By        Hon'ble Mr/Mrs Justice PRATIBHA RANI J 
Petitioner/      
Panes             TINKKU RAM 
Respondent/ 
Defendant Name    STATE OF THE NCT OF DELHI 
Petitioner's/ 
Plaintiff's Adv   S B DSNDAPANL 
Respondent's/ 
Defendant's Adv   SALEEM AHMED 
Print Date        22-Jul-2015 
Company Name      RICOH INDIA LTD. 

---page_breaker---

               W.P (Crl.) 132 2013 

Recvived the enclosed jail petition of Sh.Tinku Ram S/o Sh.Saltanti 
from Central Jail Tihar New Delhi, through Delhi High Court Legal
Services Committee, New Delhi vide letter No.215 dated 22.01.2013 

Through this jail petition above named petitioner is praying for 
parole on the ground mentioned in the petition. 

May the petition for the parole be registered as W.P.(Crl.)  /2013 
and be placed before appropriate bench of this court. 

AOJ/(Crl)

IN THE HIGH COURT OF DELHI AT NEW DELHI 
Filing No. 11719/2013 

THE MATTER OF 

TINKU RAM                        Petitioner 
                Versus 
STATE                            Respondent 

No Connecting matter found. 

Filing Assistant Kuldeep Kumar 


AOJ/FILING 
"""

questions = """
1: LAWS INVOLVED
2: Who is the informant or complainant
3: What is the name of the victim
4: What is the date or of the commitment of the offence
5: What is the place of occurrence of the offence
6: How much time was taken by the police to reach the place of occurrence after
the receipt of information of commission of the offence
7: What is the name of the accused arrested at the spot
8: What was the manner of assault
9: What is the description of the weapon used during the occurrence of the offence
10: What is the description of the specific overt act performed by of the accused
person(s)
11: Who identified the body of the deceased
12: What was the statement of the accused person(s) under section 313 Cr.P.C on
examination.
"""


prompt = """For the below case file answer these questions. 
Questions:{questions}"
Case File:{case_text}

Format the output as JSON with the question serial number as key and the answer as the value
"""

In [48]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(prompt)
print(prompt_template)

messages = prompt_template.format_messages(questions=questions, case_text=case_text)
chat = ChatOpenAI(temperature=0.0, model=llm_model)
response = chat(messages)
print(response.content)
print(type(response.content))

input_variables=['case_text', 'questions'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['case_text', 'questions'], template='For the below case file answer these questions. \nQuestions:{questions}"\nCase File:{case_text}\n\nFormat the output as JSON with the question serial number as key and the answer as the value\n'))]


ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
# We need to convert this output from string format to json format

from langchain.output_parsers import ResponseSchema, StructuredOutputParser


